<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/06_combined_model_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-06-30 23:53:45--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11845 (12K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]  11.57K  --.-KB/s    in 0s      

2022-06-30 23:53:45 (103 MB/s) - ‘utils.py’ saved [11845/11845]



In [2]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [3]:
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

!pip install -q tensorflow-io
import tensorflow_io as tfio

from utils import get_train_val_ds, get_class_weight

     |████████████████████████████████| 25.9 MB 1.6 MB/s 


In [7]:
BATCH_SIZE = 32
IMG_SIZE = (256, 256)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b3/feature_vector/2'

In [8]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [9]:
train_ds, val_ds = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE, img_size=IMG_SIZE)

Found 17043 files belonging to 16 classes.
Found 3495 files belonging to 16 classes.


In [10]:
classes, class_weight = get_class_weight()

In [11]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

val_ds = (
    val_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [12]:
rgb_input = keras.Input(shape=IMG_SIZE+(3,))
rgb_hub_module = hub.KerasLayer(HUB_URL,  trainable=True)
xyz_hub_module = hub.KerasLayer(HUB_URL,  trainable=True)

rgb_features = rgb_hub_module(rgb_input)

xyz_input = tfio.experimental.color.rgb_to_xyz(rgb_input)
xyz_features = xyz_hub_module(xyz_input)

features = keras.layers.Concatenate()([rgb_features, xyz_features])

output = keras.layers.Dense(N_CLASSES, activation='softmax')(features)

model = keras.models.Model(inputs=[rgb_input], outputs=[output])

In [13]:
model._name = 'combined-model-v2'

In [14]:
metrics = [
            keras.metrics.AUC(name='auc', curve='PR', num_thresholds=100),
            'accuracy'
          ]

loss_fn = keras.losses.CategoricalCrossentropy()
lr_schedule =tf.keras.optimizers.schedules.ExponentialDecay(1e-3, 500, .9)

model.compile(loss=loss_fn,
            optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
            metrics=metrics)

In [15]:
early_stopping = keras.callbacks.EarlyStopping(
                                              monitor='val_accuracy', 
                                              verbose=1,
                                              patience=6,
                                              mode='max')

check_point_path = os.path.join('./logs/checkpoints', model._name)
check_point = keras.callbacks.ModelCheckpoint(
                                            filepath=check_point_path,
                                            monitor='val_accuracy',
                                            save_best_only=True,
                                            mode='max')
        
callbacks = [early_stopping, check_point]

In [16]:
history = model.fit(train_ds,
                    callbacks=callbacks,
                    epochs=15,
                    validation_data=val_ds,
                    class_weight=class_weight)

Epoch 1/15
533/533 [==============================] - ETA: 0s - loss: 16.7885 - auc: 0.7577 - accuracy: 0.6773

INFO:tensorflow:Assets written to: ./logs/checkpoints/combined-model-v2/assets


INFO:tensorflow:Assets written to: ./logs/checkpoints/combined-model-v2/assets


533/533 [==============================] - 2526s 5s/step - loss: 16.7885 - auc: 0.7577 - accuracy: 0.6773 - val_loss: 0.9865 - val_auc: 0.7509 - val_accuracy: 0.6801
Epoch 2/15
533/533 [==============================] - ETA: 0s - loss: 9.5662 - auc: 0.8896 - accuracy: 0.8095

INFO:tensorflow:Assets written to: ./logs/checkpoints/combined-model-v2/assets


...

INFO:tensorflow:Assets written to: ./logs/checkpoints/combined-model-v2/assets


533/533 [==============================] - 485s 909ms/step - loss: 0.6967 - auc: 0.9963 - accuracy: 0.9833 - val_loss: 0.9507 - val_auc: 0.8631 - val_accuracy: 0.8361
Epoch 13/15
533/533 [==============================] - ETA: 0s - loss: 0.5747 - auc: 0.9969 - accuracy: 0.9852

INFO:tensorflow:Assets written to: ./logs/checkpoints/combined-model-v2/assets


INFO:tensorflow:Assets written to: ./logs/checkpoints/combined-model-v2/assets


533/533 [==============================] - 484s 907ms/step - loss: 0.5747 - auc: 0.9969 - accuracy: 0.9852 - val_loss: 1.0763 - val_auc: 0.8458 - val_accuracy: 0.8386
Epoch 14/15
533/533 [==============================] - 450s 842ms/step - loss: 0.5076 - auc: 0.9980 - accuracy: 0.9870 - val_loss: 0.9945 - val_auc: 0.8529 - val_accuracy: 0.8335
Epoch 15/15
533/533 [==============================] - 449s 841ms/step - loss: 0.4320 - auc: 0.9979 - accuracy: 0.9900 - val_loss: 0.9889 - val_auc: 0.8552 - val_accuracy: 0.8361


In [17]:
!cp  -r /content/logs/checkpoints/combined-model-v2 /content/drive/MyDrive/checkpoints/combined-v2